In [1]:
import csv
import json
import numpy as np
import pandas as pd

In [2]:
path = '/nfs/research/regan/data_propara/data/corenlp_fd_physical_completed_v2.json'

augmented_data_path = '/nfs/research/regan/src/coling2020-code/data/augmented_data_for_image_schema_to_whitelist.csv'

#augmented_data_path = '/nfs/research/regan/src/coling2020-code/data/augmented_with_ccs_rq13_v3.csv'

In [3]:
with open(path, 'r') as f:
    data = json.load(f)
len(data); data["664"]

{'vnclass': 'bump-18.4',
 'syntax': 'Sbj V PathP',
 'fd1': 'Physical',
 'fd2': 'Force',
 'fd3': '',
 'example': 'The grocery cart hit against the wall',
 'domain': 'Physical',
 'pred_calc': 'INTL(grocery cart) & FRC(grocery cart,wall)',
 'features': [{'text': 'The',
   'dep': 'det',
   'head': '3',
   'lemma': 'the',
   'children': [],
   'index': '1',
   'type': 'O',
   'pos': 'DT'},
  {'text': 'grocery',
   'dep': 'compound',
   'head': '3',
   'lemma': 'grocery',
   'children': [],
   'index': '2',
   'type': 'O',
   'pos': 'NN'},
  {'text': 'cart',
   'dep': 'nsubj',
   'head': '4',
   'lemma': 'cart',
   'children': ['The', 'grocery'],
   'index': '3',
   'type': 'ARTIFACT',
   'pos': 'NN',
   'q_value': 'INTL',
   'fd_relation': 'FRC',
   'fd_subsequent': '7',
   'subevent_aspect': 'CycAch'},
  {'text': 'hit',
   'dep': 'ROOT',
   'head': '0',
   'lemma': 'hit',
   'children': ['cart', 'wall'],
   'index': '4',
   'type': 'O',
   'pos': 'VBD',
   'vnclass': '18.4'},
  {'text': 'a

In [4]:
for k, v in data.items():
    if v['example']=='My computer connected to his computer':
        ent_idx = 0
        ex = v['example'].split()
        for idx, feat in enumerate(v['features']):
            if feat['text']=='computer':
                feat['text']+=str(ent_idx)
                ex[idx] += str(ent_idx) 
                ent_idx+=1
        v['example'] = ' '.join(ex)            

In [5]:
def get_augmented_data(path):
#     with open(path, 'r') as fp:
#         reader = csv.reader(fp, delimiter=",", quotechar='"')
#         data_read = [row for row in reader]
    df_augment = pd.read_csv(path)
    return df_augment

df_augment = get_augmented_data(augmented_data_path)

df_augment.tail(100)

,pk,example,good
23885,433.2,Parity of pit while salting,0
23886,433.2,Holing hole in the bunker,0
23887,433.2,Closing of the hole in the dirt,0
23888,433.2,Crossing the Land,0
23889,433.2,Trouble in the pit in the dirt,0
...,...,...,...
23980,433.4,Putting water on seeds and bilge,0
23981,433.4,Place Water on Seeds and Holdings,0
23982,433.4,Pour water on seeds and holds for population f...,0
23983,433.4,Remove fish from seeds and hold.,0


In [6]:
def create_causal_chain(features):

    for e in features:
        if e["dep"] == "ROOT":
            ents_list = e["children"]
            root_index = e["index"]
            
    entities = []
    for e in features:
        #if e["text"] in ents_list and e["head"] == root_index:
        if "q_value" in e or "fd_relation" in e:
            entity = {"text": e["text"], "q_value": e["q_value"], "index": int(e["index"]), "dep": e["dep"]}
            if "fd_subsequent" in e:
                if e["fd_subsequent"] != "":
                    entity["fd_subsequent"] = int(e["fd_subsequent"])
                    try:
                        entity["fd_relation"] = e["fd_relation"]
                    except KeyError:
                        print(e['text'])
                elif "fd_relation" in e:
                    entity["fd_relation"] = "None" 
                    entity["fd_relation"] = "None"
            else:
                entity["fd_relation"] = "None"

            entities.append(entity)
                
    def get_preceding_element_in_causal_chain(entities, idx): 
        for e in entities:
            if e["fd_subsequent"] == idx:
                next_index = e["index"]
                return next_index, e
        return None, e
        
                
    ordered_causal_chain = []
    all_indices = []
    for entity in entities:
        if 'fd_subsequent' not in entity or entity['fd_subsequent']==-1:
            ordered_causal_chain.append(entity)
            entities.remove(entity)
            all_indices.append(entity["index"])
    while all_indices:
        idx = all_indices.pop(0)
        
        for entity in entities:
            if entity not in ordered_causal_chain:
                next_index, ele = get_preceding_element_in_causal_chain(entities, idx)
                if next_index:
                    all_indices.append(next_index)
                    
                if ele not in ordered_causal_chain:
                    ordered_causal_chain.insert(0, ele)
    return ordered_causal_chain

def get_root_verb(features):
    
    verb = 'be'
    for f in features:
        if f["head"] == "0":
            verb = f["lemma"]      
    return verb

def get_fd(v):
#     fd = v["fd1"] + " " + v["fd2"]
#     if v["fd3"] != "":
#         fd += " " + v["fd3"]
    fd1 = v['fd1']
    fd2 = v['fd2']
    if len(v['fd3'])<1:
        fd3 = 'none'
    else:
        fd3 = v['fd3']
    return fd1, fd2, fd3

all_items = []

all_examples = []

for k, v in data.items():

    cc = create_causal_chain(v["features"])
    if len(cc)==0:
        print(v['example'], v["features"])
        print()
    fd1 = v['fd1']
    fd2 = v['fd2']
    if len(v['fd3'])<1:
        fd3 = 'none'
    else:
        fd3 = v['fd3']
    item = {"pk":k, "cc": cc, "sentence": v["example"], "syntax":v["syntax"], "fd1":fd1, "fd2":fd2, "fd3":fd3,
           "verb": get_root_verb(v["features"]), 'augmented': 0 }
    
    all_examples.append(v["example"])
    all_items.append(item)
        
all_items[120]

{'pk': '772',
 'cc': [{'text': 'I',
   'q_value': 'VOL',
   'index': 1,
   'dep': 'nsubj',
   'fd_subsequent': -1,
   'fd_relation': 'None'}],
 'sentence': 'I flossed',
 'syntax': 'Sbj V',
 'fd1': 'Volitional',
 'fd2': 'Internal',
 'fd3': 'none',
 'verb': 'floss',
 'augmented': 0}

In [7]:
len(all_items)

565

In [9]:
for idx, row in df_augment.iterrows():        
    ex = row.example
    #ex = row.sentence
    if ex not in all_examples and ex+'.' not in all_examples and row.good==1:
        pk = str(row.pk)
        if pk in ['51.2', '207.3']:
            fd='Cause COS'
            fd1 = 'Cause'
            fd2 = 'COS'
            fd3 = 'none'
        elif pk in ['116.2', '116.0', '52.5']:
            fd='Cause Create'
            fd1 = 'Cause'
            fd2 = 'Create'
            fd3 = 'none'
        elif pk in ['433.4', '433.3']:
            fd='Volitional Place'
            fd1 = 'Volitional'
            fd2 = 'Place'
            fd3 = 'none'
        elif pk in ['433.2']:
            fd='Volitional Create' 
            fd1 = 'Volitional'
            fd2 = 'Create'
            fd3 = 'none'
        elif pk in ['433.1']:
            fd='Volitional Internal Purp' 
            fd1 = 'Volitional'
            fd2 = 'Internal'
            fd3 = 'Purp'
        elif pk in ['433.0']:
            fd='Volitional Internal' 
            fd1 = 'Volitional'
            fd2 = 'Internal'
            fd3 = 'none'
        elif pk in ['330.6']:
            fd='Autonomous Internal Purp' 
            fd1 = 'Autonomous'
            fd2 = 'Internal'
            fd3 = 'Purp'
        elif pk in ['321.1', '158.5', '157.3']:
            fd='Autonomous COS Purp' 
            fd1 = 'Autonomous'
            fd2 = 'COS'
            fd3 = 'Purp'
        elif pk in ['13.4']:
            fd='Physical Remove' 
            fd1 = 'Physical'
            fd2 = 'Remove'
            fd3 = 'none'
        elif pk in ['13.3']:
            fd='Autonomous Motion'
            fd1 = 'Autonomous'
            fd2 = 'Motion'
            fd3 = 'none'
        elif pk in ['7.6']:
            fd='Autonomous Place'
            fd1 = 'Autonomous'
            fd2 = 'Place'
            fd3 = 'none'
        elif pk in ['4.1', '4.2']:
            fd='Autonomous Location'
            fd1 = 'Autonomous'
            fd2 = 'Location'
            fd3 = 'none'
        elif pk in ['286.4']:
            fd='Volitional COS Purp'
            fd1 = 'Volitional'
            fd2 = 'COS'
            fd3 = 'Purp'
        else:
            pk = pk.split('.')[0]
            fd1, fd2, fd3 = get_fd(data[pk])
            
        item = {"pk":pk, "cc": np.NaN, "sentence": ex, "syntax":np.NaN, "fd1":fd1, "fd2":fd2, "fd3":fd3,
               "verb": np.NaN, 'augmented': 1 }
        
        all_items.append(item)
          

664
664
664
664
664
664
664
664
664
664
664
664
664
664
664
664
665
665
665
665
666
666
666
666
666
666
667
667
667
667
669
669
669
669
669
670
670
670
670
670
670
671
671
671
671
671
671
672
672
672
672
672
673
673
673
674
675
675
675
676
676
676
676
199
199
679
679
679
679
680
680
132
132
132
245
245
248
248
681
681
682
682
682
682
683
683
684
684
684
684
684
684
684
684
684
684
685
685
685
685
685
686
686
686
686
686
686
687
687
687
687
687
687
688
688
688
688
688
688
689
689
689
690
690
690
690
690
690
691
691
691
691
691
691
692
692
692
692
692
692
692
693
693
693
693
693
693
694
694
694
694
694
694
694
695
695
695
695
696
696
696
696
696
697
697
698
698
698
698
698
698
698
698
699
699
699
699
699
700
700
700
700
700
700
700
700
700
700
700
700
701
701
701
701
701
701
701
702
702
702
702
702
703
703
703
703
703
703
703
703
704
704
705
705
705
706
706
706
706
706
187
187
247
247
247
249
249
249
249
246
246
707
708
709
709
709
709
1170
1170
710
710
710
710
711
711
711
711
711
711
71

In [ ]:
len(all_items)

In [ ]:
# for item in all_items:
#     if len(item['cc'])==4:
#         print(len(item['cc']), item['cc'])
#         print()

cnt_part = 0
cnt_mult_qvalue = 0
for item in all_items:
    cc = item['cc']
    if cc is not np.NaN:
        for c in cc:
            cnt_part += 1
            if '::' in c['q_value']:
                cnt_mult_qvalue += 1

print("# with multiple q_values:", cnt_mult_qvalue)
print("Pct:", cnt_mult_qvalue/cnt_part)
print("Total participants:", cnt_part)

In [ ]:
df = pd.DataFrame(all_items)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df.tail(20)

In [ ]:
for idx, row in df.iterrows():
    if row.augmented == 1:
        pk = row.pk
        if '.' not in pk:
            cc = df[(df.pk == pk) & (df.augmented == 0)].cc.values[0]
            df.iloc[idx].cc = cc

In [ ]:
df[df['augmented']==1]

In [ ]:
df[df.sentence=='Pennie roasted the things']

In [ ]:
df.groupby('fd1').count().pk

In [ ]:
df.groupby('augmented').count().pk

In [ ]:
df.groupby('fd3').count().pk

In [ ]:
df.groupby('fd2').count().pk

In [ ]:
df[df['fd2']=='Deprive'].head(25)

In [ ]:
#df.to_pickle("../data/fd_vn_physical_with_augmented_v6.pkl")